# 每日简报

In [1]:
import os
import time
import datetime
import numpy as np
import pandas as pd

## 第一步：加载合同报表内容到内存

In [2]:
# 参数
PATH = os.path.abspath(r"C:\Users\Administrator\Downloads")
CONTRACT_FILENAME = "7月合同报表.xls"

In [3]:
# 读取当月月报
df = pd.DataFrame(pd.read_excel(os.path.join(PATH,CONTRACT_FILENAME),header =1))
# df.head()

In [4]:
companys = df['卖家'].unique().tolist()

for co in companys:
    if co.__contains__("淮钢"):
        print("今天又淮钢数据，模板中没有，是否需要统计，请教严丰")



## 第2步：转换df的“合同日期”时间，用于分别计算当日和当月的数据

In [5]:
def format_date(test_date_string):
    date = datetime.datetime.strptime(test_date_string,"%Y-%m-%d %H:%M:%S")
    return date
df["合同日期"] = df["合同日期"].map(format_date)

In [6]:
localtime = time.localtime(time.time())
def compare_date(contract_time):
    # “合同日期”取出 年，月，日；与本地时间的 年 月 日进行比较，用于取出合同日期为当天的
    contract_year = contract_time.date().year
    contract_month = contract_time.date().month
    contract_day = contract_time.date().day
    
    localtime = time.localtime(time.time())
    if contract_year == localtime.tm_year and contract_month == localtime.tm_mon and contract_day == localtime.tm_mday: 
    # if contract_year == localtime.tm_year and contract_month == localtime.tm_mon and contract_day == 11:
        return True
    else:
        return False


# 用 compare_date方法判断 合同日期“是否是当天”，将该列作为后续计算当天数据指标的判断条件
df["是否是当天"] = df["合同日期"].map(compare_date)

## 第3步：根据“是否是当天”字段进行条件筛选，将获取到的当天的数据进行汇总，求和，取出当天的计算数据。

In [7]:
df["是否是当天"].loc[df["是否是当天"] == True]

756    True
757    True
758    True
759    True
760    True
       ... 
935    True
936    True
937    True
938    True
939    True
Name: 是否是当天, Length: 184, dtype: bool

## 当日数据
- 当天 平台钢材成交总量   **today_steel_transaction_count**
    - 当天 合同【数量】字段汇总
- 当日现货竞拍成交总量   **today_bids_transaction_count**
        - 当日物贸余材竞拍成交量 + 当日轧花厂竞拍成交量 + 当日东北特钢竞拍成交量
    - 物贸余材竞拍成交总量 **today_wumao_bids_transaction_count**
        - 卖家是【江苏沙钢物资贸易有限公司】的【数量】字段汇总
    - 轧花厂竞拍成交总量 **today_yahuacang_bids_transaction_count**
        - 卖家是【张家港市锦丰轧花剥绒有限责任公司】的【数量】字段汇总
    - 东北特钢竞拍成交总量  **today_dongbei_bids_transaction_count**
        - 卖家是【东北特殊钢集团股份有限公司】的【数量】字段汇总
- 期货挂牌成交 **qihuo_guapai_count**
    - 卖家是【江苏沙钢物资贸易有限公司1】，产品是["优线", "普冷","酸洗","轧硬"]
- 期货撮合成交 **qihuo_cuohe_count**
    - 卖家是【江苏沙钢物资贸易有限公司1】，产品是["花纹卷", "普卷","C料","螺纹","盘螺"]
- 现货竞拍增值
- 平均吨钢竞拍增值



In [8]:
# 当天 平台钢材成交总量 today_steel_transaction_count
# 当天 合同【数量】字段汇总
today_steel_transaction_count = df["数量"].loc[df["是否是当天"] == True].sum()
today_steel_transaction_count

2000.877

In [9]:
today_bids_transaction_count = 0
today_wumao_bids_transaction_count = df["数量"].loc[(df["是否是当天"] == True) & (df["卖家"] == "江苏沙钢物资贸易有限公司")].sum()
today_yahuacang_bids_transaction_count = df["数量"].loc[(df["是否是当天"] == True) & (df["卖家"] == "张家港市锦丰轧花剥绒有限责任公司")].sum()
today_dongbei_bids_transaction_count = df["数量"].loc[(df["是否是当天"] == True) & (df["卖家"] == "东北特殊钢集团股份有限公司")].sum()


today_bids_transaction_count = today_wumao_bids_transaction_count + today_yahuacang_bids_transaction_count + today_dongbei_bids_transaction_count

In [10]:
guapai_today = df.query('品名 == ["优线", "普冷","酸洗","轧硬"]').loc[(df["卖家"] == "江苏沙钢物资贸易有限公司1") & (df["是否是当天"] == True)]
qihuo_guapai_count = guapai_today["数量"].sum()
cuohe_today = df.query('品名 == ["花纹卷", "普卷","C料","螺纹","盘螺"]').loc[(df["卖家"] == "江苏沙钢物资贸易有限公司1") & (df["是否是当天"] == True)]
qihuo_cuohe_count =  cuohe_today["数量"].sum()

#### 用于汇总表统计

In [11]:
print(today_steel_transaction_count)

total_price = df["贷款"].loc[(df["是否是当天"] == True)].sum()
print(total_price)

2000.877
7356274.649999999


## 本月累计数据

## 当月数据
- 本月 平台钢材成交总量 **month_steel_transaction_count**
    - 竞拍成交总量   **month_bids_transaction_count**
    - 期货挂牌成交总量  **month_qihuo_transaction_count**
    - 竞拍撮合成交总量  **month_jingpai_cuohe_transaction_count**
    - 轧花厂成交总量   **month_yahuacang_transaction_count**
- 本月竞拍增值
- 平均吨钢竞拍增值

### 注意：
需要注意期货挂牌成交 和 期货撮合成交 的计算，需要和每日导出的采购成交一览表一起对比。   
由于里面产品是不一定会挂在挂牌 或 撮合 的。

In [12]:
# 本月 平台钢材成交总量 month_steel_transaction_count
month_steel_transaction_count = df["数量"].sum()

# 本月累计竞拍成交总量
month_bids_transaction_count = df["数量"].loc[df["卖家"] != "江苏沙钢物资贸易有限公司1"].sum()
# 期货挂牌成交总量
qihuo = df.query('品名 == ["优线", "普冷","酸洗","轧硬"]').loc[df["卖家"] == "江苏沙钢物资贸易有限公司1"]
month_qihuo_transaction_count = qihuo['数量'].sum()
# 竞拍撮合成交总量
cuohe = df.query('品名 == ["花纹卷", "普卷","C料","螺纹","盘螺"]').loc[df["卖家"] == "江苏沙钢物资贸易有限公司1"]
month_jingpai_cuohe_transaction_count = cuohe["数量"].sum()
# 轧花厂成交总量
month_yahuacang_transaction_count = df["数量"].loc[df["卖家"] == "张家港市锦丰轧花剥绒有限责任公司"].sum()

## 电商交易日简报 拼接

In [13]:
# 本日内容拼接
daily_report = """电商交易日简报：
{0}月{1}日，电商平台钢材成交总量（{2}）吨。
本日现货竞拍成交（{3}）吨，其中物贸余材竞拍成交（{4}）吨；轧花厂竞拍成交（{5}）吨；东北特钢竞拍成交（{6}）吨。期货挂牌成交（{12}）吨；期货撮合成交（{13}）吨。本日现货竞拍增值（_____）万元，平均吨钢竞拍增值（_______）元/吨。
本月截至今日，电商平台钢材成交总量（{7}）吨。其中竞拍成交（{8}）吨；期货挂牌成交（{10}）吨；竞拍撮合成交（{9}）吨；轧花厂成交（{11}）吨。本月竞拍增值（________）万元，平均吨钢竞拍增值（______）元/吨。  
""".format(localtime.tm_mon,
           localtime.tm_mday,
           round(today_steel_transaction_count,2),
           round(today_bids_transaction_count,2),
           round(today_wumao_bids_transaction_count,2),
           round(today_yahuacang_bids_transaction_count,2),
           round(today_dongbei_bids_transaction_count,2),
           round(month_steel_transaction_count,2),
           round(month_bids_transaction_count,2),
           round(month_jingpai_cuohe_transaction_count,2),
           round(month_qihuo_transaction_count,2),
           round(month_yahuacang_transaction_count,2),
           round(qihuo_guapai_count,2),
           round(qihuo_cuohe_count,2)
          
          
          )

In [14]:
daily_report

'电商交易日简报：\n7月3日，电商平台钢材成交总量（2000.88）吨。\n本日现货竞拍成交（2000.88）吨，其中物贸余材竞拍成交（1700.88）吨；轧花厂竞拍成交（300.0）吨；东北特钢竞拍成交（0.0）吨。期货挂牌成交（0.0）吨；期货撮合成交（0.0）吨。本日现货竞拍增值（_____）万元，平均吨钢竞拍增值（_______）元/吨。\n本月截至今日，电商平台钢材成交总量（6674.34）吨。其中竞拍成交（6674.34）吨；期货挂牌成交（0.0）吨；竞拍撮合成交（0.0）吨；轧花厂成交（600.0）吨。本月竞拍增值（________）万元，平均吨钢竞拍增值（______）元/吨。  \n'

In [15]:
filename = '每日简报.txt'
with open(os.path.join(PATH,filename),'w') as f: # 如果filename不存在会自动创建， 'w'表示写数据，写之前会清空文件中的原有数据！
    f.write(daily_report)

电商交易日简报：
5月30日，电商平台钢材成交总量（1616.15）吨。
本日现货竞拍成交（1616.15）吨，其中物贸余材竞拍成交（1456.15）吨；轧花厂竞拍成交（160.0）吨。本日现货竞拍增值1.82万元，平均吨钢

竞拍增值12.49元/吨。
本月截至今日，电商平台钢材成交总量（200619.63）吨。其中竞拍成交（83727.11）吨；期货挂牌成交（18600.0）吨；竞拍撮合成交（22100.0

）吨；轧花厂成交（10611.0）吨；宽厚板指数定价成交量（76192.52）吨。本月竞拍增值270.16万元，平均吨钢竞拍增值38.53元/吨。  
本日电商平台注册新客户（4）家，本月累计注册新客户（90）家，其中销售类客户（16）家，供应类客户（74）家。 
本日发布标书（23）份，本月累计发布标书（544）份。
本日系统运行正常。